<a href="https://colab.research.google.com/github/juanignaciogiacobbe/7506R-2C2023-GRUPO15/blob/main/7506R_TP1_GRUPO15_CHP2_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 1: Reservas de Hotel</center>
### <center> Grupo número 15</center>
### <center> Giacobbe, Juan Ignacio; Olaran ,Sebastian; Ayala, Tomás</center>

#**Checkpoint 2**: Árbol de decisión


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las librerías que vamos a utilizar

In [39]:
#Para análisis de los datos
import pandas as pd
import numpy as np

#Para visualizaciones
import seaborn as sns
from matplotlib import pyplot as plt

Cargamos los archivos csv de datos en dos DataFrames de Pandas(importamos para el dataset de train el dataset que nos quedó como resultado en el checkpoint 1)

In [40]:
ds_hoteles_test = pd.read_csv('./hotels_test.csv')
ds_hoteles_train = pd.read_csv('./hoteles_train_final.csv')

Comenzamos haciendo un breve repaso de las características del dataset a analizar

In [41]:
ds_hoteles_train.shape

(60897, 32)

Tenemos un dataset con 60897 registros de reservas de hotel, con 32 columnas correspondientes a variables que describen las reservas(por ejemplo, tenemos una variable que indica la cantidad de adultos que tenemos por reserva).


Veamos dos registros del dataset

In [21]:
ds_hoteles_train.head(2)

,Unnamed: 0,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,id,is_canceled
0,0,City Hotel,49,2016,September,37,5,1,2,1,...,No Deposit,9.0,-1.0,0,Transient,115.5,0,1,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,1,Resort Hotel,4,2015,October,44,31,0,1,2,...,No Deposit,-1.0,-1.0,0,Transient,42.0,1,0,b1428f80-c56c-4ae4-91a9-6962edae08b1,0


Veamos las variables que hay en el dataset

In [22]:
ds_hoteles_train.columns.tolist()

['Unnamed: 0',
 'hotel',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'id',
 'is_canceled']

**Descripción de cada campo**



*   hotel(categórica): indica el tipo de hotel en el que se realizó la reserva.
*   lead_time(cualitativa): indica el tiempo desde que se reservó hasta que la persona llegó al hotel.
*   arrival_date_year/month/week_number/day_of_month: indican la fecha de llegada.
*   stays_in_weekend/week_nights: indican las noches que están reservadas.
*   adults, children, babies: indican la cantidad de adultos, niños y bebés respectivamente.
*   meal: indica el tipo de comida que incluye la reserva.
*   country: el país destino de la reserva.
*   market_segment:
*   distribution_chanel:
*   is_repeated_guest:












##Transformación de variables

Vemos que, tenemos bastantes variables categóricas(como la variable hotel), y a la hora de implementar el árbol de decisión vamos a necesitar solamente variables cualitativas. Como queremos que las variables categóricas formen parte de nuestras variables predictoras, vamos a aplicarles transformaciones.

In [101]:
#Creamos una copia del dataset de train
ds_trabajo = ds_hoteles_train.copy()
ds_trabajo.head()

,Unnamed: 0,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,id,is_canceled
0,0,City Hotel,49,2016,September,37,5,1,2,1,...,No Deposit,9.0,-1.0,0,Transient,115.5,0,1,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,1,Resort Hotel,4,2015,October,44,31,0,1,2,...,No Deposit,-1.0,-1.0,0,Transient,42.0,1,0,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
2,2,City Hotel,25,2016,October,42,13,0,1,2,...,No Deposit,9.0,-1.0,0,Transient,229.0,0,2,2226c018-ea20-4c4e-8f71-ea66aee70d7c,0
3,3,City Hotel,26,2016,December,51,12,2,5,2,...,Non Refund,154.0,-1.0,0,Transient,75.0,0,0,314f56dc-2e4d-4298-9bdd-27bb008bffff,1
4,4,City Hotel,104,2017,March,12,19,2,5,2,...,No Deposit,20.0,-1.0,0,Transient-Party,75.0,0,1,28c75445-c603-413a-babf-ad377018b592,0


Para las variables hotel, meal, market_segment, distribution_channel, reserved_room_type, assigned_room_type y customer_type vamos a tranformarlas utilizando One Hot Encoding. De esta manera, crearemos variables "dummies" que nos brinden la misma información que estas variables, con la ventaja de convertirlas en variables numéricas y así poder utilizarlas para nuestros modelos predictivos

In [102]:
columnas_a_transformar = ['hotel', 'meal', 'market_segment', 'distribution_channel', 'customer_type']

ds_trabajo = pd.get_dummies(data=ds_trabajo, columns=columnas_a_transformar, drop_first=True)
ds_trabajo.head()

,Unnamed: 0,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,49,2016,September,37,5,1,2,1,0.0,...,0,1,0,0,0,1,0,0,1,0
1,1,4,2015,October,44,31,0,1,2,0.0,...,0,0,0,1,0,0,0,0,1,0
2,2,25,2016,October,42,13,0,1,2,2.0,...,0,1,0,0,0,1,0,0,1,0
3,3,26,2016,December,51,12,2,5,2,0.0,...,0,0,0,0,0,1,0,0,1,0
4,4,104,2017,March,12,19,2,5,2,0.0,...,1,0,0,0,0,1,0,0,0,1


Veamos que esas variables se transformaron correctamente. Ahora, nos quedó pendiente transformar las variables country, arrival_date_month, assigned_room_type y reserved_room_type. La razón por la cual no utilizamos one hot encoding para transformarlas es porque nos iba a generar demasiadas columnas nuevas(tengamos en cuenta que solo para country se iban a generar más de 100 columnas nuevas), agregando mucha complejidad computacional con poco beneficio. Para estas variables, vamos a enumerar sus valores(por ejemplo, los meses de llegada, en vez de tener valores como "enero" y "febrero", serán 1 y 2 por ejemplo). De esta manera, las transformaremos para poder usarlas como variables predictivas para nuestros modelos.

Vamos a usar el método factorize de pandas, que nos permite codificar el objeto como un tipo enumerado o variable categórica.

In [105]:
ds_trabajo['country'] = pd.factorize(ds_trabajo['country'])[0]
ds_trabajo['assigned_room_type'] = pd.factorize(ds_trabajo['assigned_room_type'])[0]
ds_trabajo['reserved_room_type'] = pd.factorize(ds_trabajo['reserved_room_type'])[0]
ds_trabajo['arrival_date_month'] = pd.factorize(ds_trabajo['arrival_date_month'])[0]

ds_trabajo.head()

,Unnamed: 0,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,49,2016,0,37,5,1,2,1,0.0,...,0,1,0,0,0,1,0,0,1,0
1,1,4,2015,1,44,31,0,1,2,0.0,...,0,0,0,1,0,0,0,0,1,0
2,2,25,2016,1,42,13,0,1,2,2.0,...,0,1,0,0,0,1,0,0,1,0
3,3,26,2016,2,51,12,2,5,2,0.0,...,0,0,0,0,0,1,0,0,1,0
4,4,104,2017,3,12,19,2,5,2,0.0,...,1,0,0,0,0,1,0,0,0,1


Con esto, hemos tranformado y preparado el dataset para comenzar con nuestros modelos predictivos, ya que ahora todas nuestras variables van a funcionar como variables predictoras.